In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import numpy as np
from math import radians, degrees
import pandas as pd

In [ ]:
# 1-Butanol (g-mm-s):
R = 18 # mm
rho = 0.8098/1000 # g/mm^3
g = 9.81*1000 # mm/s^2
sigma = 24.93 # g/s^2
delta = 1e-9
h = 0.001 # step

In [ ]:
def f(s, y, R0):
    return np.array((np.cos(y[1]), 2/R0-rho*g*y[2]/sigma-np.sin(y[1])/y[0], -np.sin(y[1])))

In [ ]:
def runge_kutta(R0):
    y0 = np.array((R0*delta, delta, 0))
    y = y0.copy()
    sk = 0
    while y0[0] < R:
        sk += h
        p1 = f(sk, y0, R0)
        p2 = f(sk+h/2, y0+h*p1/2, R0)
        p3 = f(sk+h/2, y0+h*p2/2, R0)
        p4 = f(sk+h, y0+h*p3, R0)
        y1 = np.copy(y0) + h*(p1 + 2*p2 + 2*p3 + p4)
        y0 = y1
        y = np.vstack((y, y0))  
    s_max = sk
    
    return y, s_max

In [ ]:
def m_bisection(theta_0, R1, R2, eps):
    R3 = (R1 + R2)/2
    theta_1 = runge_kutta(R1)[0][-1][1]
    theta_2 = runge_kutta(R2)[0][-1][1]
    theta_3 = runge_kutta(R3)[0][-1][1]
    while abs(degrees(theta_3 - theta_0)) > eps:
        if (theta_3 < theta_0) and (theta_3 > theta_1):
            R1 = R3
        else:
            R2 = R3
        R3 = (R1 + R2)/2
        theta_3 = runge_kutta(R3)[0][-1][1]
    return R3, degrees(theta_3)

In [ ]:
R0, theta = m_bisection(radians(30), 15, 48000, 2)
R0, theta

In [ ]:
y = runge_kutta(R0)

In [ ]:
# r(z)
plt.plot(y[0][:, 2], y[0][:, 0])

In [ ]:
def bond_number(h0, R, theta):
    return rho*g*h0*R/(2*sigma*np.sin(theta))

In [ ]:
bond_number(abs(y[0][-1][2]), y[0][-1][0], y[0][-1][1])

In [ ]:
def m_simpson(y, a, b, n):
    m = 0
    hs = (b - a)/(2*n)
    s = (y[a][0]**2)*np.sin(y[a][1]) + 4*(y[a+hs*(2*n-1)][0]**2)*np.sin(y[a+hs*(2*n-1)][1]) + (y[b][0]**2)*np.sin(y[b][1])
    for i in range(1, n):
        s += 2*(y[a+hs*(2*i)][0]**2)*np.sin(y[a+hs*(2*i)][1]) + 4*(y[a+hs*(2*i-1)][0]**2)*np.sin(y[a+hs*(2*i-1)][1])
    return np.pi*rho*hs*s/3

In [ ]:
m_simpson(y[0], 0, y[1], 10)

In [ ]:
x = np.column_stack((np.ones(len(y[0])/4+1), np.arange(1, len(y[0])/4+1), y[0][::4,2], y[0][::4,0], np.zeros(len(y[0])/4+1)))

In [ ]:
df = pd.DataFrame(data=x)
df

In [ ]:
np.savetxt(r'drop_shape.txt', df, fmt='%f', newline='\r\n', delimiter='\t')

In [ ]:
plt.plot(df[2], df[3])